# No modificar, todo funciona

In [2]:
# Librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import chisquare, pearsonr, ks_2samp
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import parametros as p
import os
import json
import pickle

In [3]:
# Nuevos datos con requerimientos de las llegadas
with open('../2. data/llegadas.pkl', 'rb') as file:
    llegadas_n = pickle.load(file)

In [4]:
# Paso el archivo de llegadas al mismo formato que usaba la funcion datos_llegadas
new_rows = []
requerimiento_dict = {1: "OR", 2: "ICU", 3: "SDU_WARD"}
"""
# nombre_unidades: nombre de las unidades
dict_unidades = {
    "OR": 1,
    "ICU": 2,
    "SDU/WARD": 3,
    "GA": 4,
    "ED": 5
}"""
# Paso el archivo de llegadas al mismo formato que usaba la funcion datos_llegadas
ciclos = len(llegadas_n["WL"])
for llegada in ["WL", "ED"]:
    for ciclo in range(1, ciclos + 1):
        for requerimiento in range(1, 4): # 1: "OR", 2: "ICU", 3: "SDU_WARD"
                    if llegada == "WL":
                        for grd in range(5,9):
                            for repeticiones in range(llegadas_n[llegada][ciclo][(grd, requerimiento)]):
                                new_row = {
                                    'MS_GRD': grd,
                                    'TI': ciclo * 12,
                                    'HOSPITAL': 0, # hospital cero no existe, pero es para poder iterar sin errores
                                    'LLEGADA': llegada,
                                    'REQUERIMIENTO': requerimiento_dict[requerimiento]
                                }
                                new_rows.append(new_row)
                    elif llegada == "ED":
                        for hospital in range(1, 4):
                            for grd in range(1,5):
                                for repeticiones in range(llegadas_n[llegada][ciclo][(hospital, grd, requerimiento)]):
                                    new_row = {
                                        'MS_GRD': grd,
                                        'TI': ciclo * 12,
                                        'HOSPITAL': hospital,
                                        'LLEGADA': llegada,
                                        'REQUERIMIENTO': requerimiento_dict[requerimiento]
                                    }
                                    new_rows.append(new_row)

# Crear un nuevo DataFrame con las nuevas filas
new_llegadas = pd.DataFrame(new_rows)
                    

In [5]:
def calculate_kde_metrics(y_true, kde):
    
    # Calculo las métricas para el kde (que es el unico que voy a usar)
    def get_metrics(y_true, y_pred):
        # prediccion = y_pred * (y_true.sum() / y_pred.sum())  # reescala predicciones para el test
        # chi2 = chisquare(f_obs=y_true, f_exp=prediccion)
        chi2 = chisquare(f_obs=y_true, f_exp=y_pred)
        ks_stat = ks_2samp(y_true, y_pred, method = "asymp")
        cc = pearsonr(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)
        rmse = mean_squared_error(y_true, y_pred, squared=False)
        mae = mean_absolute_error(y_true, y_pred)

        return {
            'Chi2': {"Value": chi2.statistic, "p-value": chi2.pvalue},
            'KS': {"Value": ks_stat.statistic, "p-value": ks_stat.pvalue},
            'CC': {"Value": cc.statistic, "p-value": cc.pvalue},
            'R2': {"Value": r2, "p-value": "NA"},
            'RMSE': {"Value": rmse, "p-value": "NA"},
            'MAE': {"Value": mae, "p-value": "NA"}
        }
    metrics = get_metrics(y_true, kde * y_true.sum())

    return metrics

# Defino la funcion para el ajuste de los datos (la modifico para que incluya hospitales)
def datos_llegadas(llegadas, requerimiento, grd, hospital):
    # Para visualizar los datos de llegadas (cambiar requerimiento y grd que se quiera ver)
    v1 = llegadas[(llegadas["REQUERIMIENTO"] == requerimiento) & (llegadas["MS_GRD"] == grd) & (llegadas["HOSPITAL"] == hospital)]
    vector = v1["TI"].value_counts().reset_index().sort_values(by="TI")
    vector.columns = ["TI", "OCURRENCIAS"]
    v2 = vector["OCURRENCIAS"].value_counts().reset_index()
    v2.columns = ["VALOR", "OCURRENCIAS"]
    v2.sort_values(by="VALOR", inplace=True)

    # Agregar la cantidad de veces que no llego nadie, osea valor 0
    ciclos_simulacion = int(llegadas["TI"].max()/12)
    # agregar nueva fila
    cero_ocurrencias = {"VALOR": 0, "OCURRENCIAS": ciclos_simulacion - v2["OCURRENCIAS"].sum()}
    v2 = pd.concat([v2, pd.DataFrame([cero_ocurrencias])], ignore_index=True)
    # Ordenar por VALOR
    v2.sort_values(by="VALOR", inplace=True)
    v2.reset_index(inplace=True)
    v2.drop(columns=["index"], inplace=True)
    return v2

# Funciones para KDE Discreto Triangular
# Discrete Triangular Kernel
def D(a1, a2, h):
    left = sum(k**h for k in range(1, a1 + 1))
    right = sum(k**h for k in range(1, a2 + 1))
    return (a1 + a2 + 1) - (a1 + 1)**(-h) * left - (a2 + 1)**(-h) * right

# Discrete Triangular Distribution Denominator Function
def DT_kernel(y, m, a1, a2, h):
    denom = D(a1, a2, h)
    if y < m - a1 or y > m + a2:
        return 0.0
    if y < m:
        return (1 - ((m - y) / (a1 + 1))**h) / denom
    elif y > m:
        return (1 - ((y - m) / (a2 + 1))**h) / denom
    else:  # y == m
        return 1 / denom

# KDE using the discrete triangular kernel
def discrete_DT_kde(data, a1=2, a2=2, h=1.0, support=None):
    if support is None:
        support = np.arange(min(data), max(data) + 1)
    kde_pmf = np.zeros_like(support, dtype=float)
    for x in data:
        for i, y in enumerate(support):
            kde_pmf[i] += DT_kernel(y, x, a1, a2, h)
    kde_pmf /= kde_pmf.sum()  # Normalize to make it a valid PMF
    return support, kde_pmf

# Search for the smallest h with p > 0.065
def find_satisfactory_h(data, empirical_counts, support, a1=1, a2=1, initial_h=0.5, p_lb=0.065, p_ub=0.1):
    p = 0
    h = initial_h
    contador = 0
    escala_h = 0.001
    while True:
        _, kde = discrete_DT_kde(data, a1=a1, a2=a2, h=h, support=support)
        expected_counts = kde * empirical_counts.sum()
        chi2, p = chisquare(empirical_counts, expected_counts)
        if p > p_lb and p < p_ub:
            return h, kde, chi2, p
        elif p > p_ub:
            h += escala_h
        elif p < p_lb:
            h -= escala_h
        # Check if h is too small
        if h < escala_h:
            return h, kde, chi2, p
        contador += 1
        if contador > 1000:
            return h, kde, chi2, p

# Main function
def ajuste_kde_triangular(vector, unidad, grd, plot, hospital, a1=1, a2=0, initial_h=1, p_lb=0.065, p_ub=0.1):
    
    # Reindexamos para que todo el soporte tenga al menos un valor llenando los ceros con 1
    values = np.arange(int(min(vector["VALOR"])), int(max(vector["VALOR"])) + 1) # Soporte para el KDE
    vector_filled = vector.set_index("VALOR").reindex(values, fill_value=1).reset_index()
    empirical_counts = vector_filled["OCURRENCIAS"].values
    los_samples = np.repeat(vector_filled["VALOR"], vector_filled["OCURRENCIAS"])
    empirical_pmf = empirical_counts / empirical_counts.sum()

    # Probar con diferentes valores de h
    best_h, final_kde_pmf, chi2, p_val = find_satisfactory_h(los_samples, empirical_counts, values, a1=a1, a2=a2, initial_h=initial_h, p_lb=p_lb, p_ub=p_ub)

    # Probar con un valor de h (descomentar si se necesita)
    # best_h = initial_h
    # _, final_kde_pmf  = discrete_DT_kde(los_samples, a1=1, a2=0, h=best_h, support=None)
    # expected_counts = final_kde_pmf * empirical_counts.sum()
    # chi2, p_val = chisquare(empirical_counts, expected_counts)
    
    # Grafico
    if plot:
        # Plot the KDE vs empirical PMF
        plt.figure(figsize=(10, 6))
        plt.bar(values - 0.2, empirical_pmf, width=0.4, label="Empirical PMF", alpha=0.7)
        plt.bar(values + 0.2, final_kde_pmf, width=0.4, label=f"DT KDE (h={best_h:.2f})", alpha=0.7)
        plt.title(f"Llegadas Empirico vs. KDE Discreto Triangular\nh: {best_h:.2f}, Chi-2: {chi2:.4f}, P-value: {p_val:.4f}")
        plt.xlabel("Cantidad de llegadas")
        plt.ylabel("Probability")
        plt.xticks(values)
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        folder = "plots/llegadas"
        filename = f"{unidad}, hospital: {hospital}, grd: {grd}, h: {best_h:.2f}, Chi-2: {chi2:.4f}, P-value: {p_val:.4f}.png"
        os.makedirs(folder, exist_ok=True)  # Create the folder if it doesn't exist
        path = os.path.join(folder, filename)
        plt.savefig(path, format='png', dpi=300)
        plt.close()
    if p_val >= p_lb:
        print(f"{unidad}, grd: {grd}, h: {best_h:.3f}, Chi-2: {chi2:.4f}, P-value: {p_val:.4f}")
        return best_h, final_kde_pmf, chi2, p_val
    else:
        print(f"{unidad}, grd: {grd}, No value of h produced a KDE with p-value > 0.065")
        return best_h, final_kde_pmf, chi2, p_val

In [6]:
resultados = {}
plot = True
for hospital in range(0, 4): # El cero es para la gente en WL
    resultados[hospital] = {}
    for requerimiento in ["OR", "ICU", "SDU_WARD"]:
            id_requerimiento = p.dict_unidades[requerimiento.replace("_","/")]
            resultados[hospital][id_requerimiento] = {}
            for grd in range(1, 9):
                resultados[hospital][id_requerimiento][grd] = {}
                v2 = datos_llegadas(new_llegadas, requerimiento, grd, hospital)
                if len(v2) > 1: # Mayor a 1 porque a todos se les agrega el cero
                    best_h, final_kde_pmf, chi2, p_val = ajuste_kde_triangular(v2, requerimiento, grd, plot, hospital, a1=1, a2=1, initial_h=0.3, p_lb=0.065, p_ub=0.1)
                    if best_h is not None:
                        vector = v2.copy()
                        # Repito este codigo para calcular las metricas
                        values = np.arange(int(min(vector["VALOR"])), int(max(vector["VALOR"])) + 1) # Soporte para el KDE
                        vector_filled = vector.set_index("VALOR").reindex(values, fill_value=1).reset_index()
                        empirical_counts = vector_filled["OCURRENCIAS"].values
                        los_samples = np.repeat(vector_filled["VALOR"], vector_filled["OCURRENCIAS"])
                        empirical_pmf = empirical_counts / empirical_counts.sum()


                        
                        resultados[hospital][id_requerimiento][grd] = {
                        "final_kde_pmf": final_kde_pmf.tolist(),
                        "best_h": best_h,
                        "metricas": calculate_kde_metrics(empirical_counts, final_kde_pmf)
                        }

OR, grd: 5, h: 0.230, Chi-2: 20.0748, P-value: 0.0657
OR, grd: 6, h: 0.141, Chi-2: 14.6580, P-value: 0.0661
OR, grd: 7, h: 0.183, Chi-2: 17.3312, P-value: 0.0674
OR, grd: 8, h: 0.095, Chi-2: 17.4176, P-value: 0.0656
ICU, grd: 5, h: 0.264, Chi-2: 20.0535, P-value: 0.0661
ICU, grd: 6, h: 0.200, Chi-2: 20.0517, P-value: 0.0661
ICU, grd: 7, h: 0.181, Chi-2: 17.4209, P-value: 0.0656
ICU, grd: 8, h: 0.090, Chi-2: 14.6763, P-value: 0.0658
SDU_WARD, grd: 5, h: 0.246, Chi-2: 14.6507, P-value: 0.0663
SDU_WARD, grd: 6, h: 0.149, Chi-2: 13.2096, P-value: 0.0672
SDU_WARD, grd: 7, h: 0.178, Chi-2: 13.2658, P-value: 0.0659
SDU_WARD, grd: 8, h: 0.077, Chi-2: 10.3327, P-value: 0.0663
OR, grd: 1, h: 0.058, Chi-2: 17.1581, P-value: 0.0709
OR, grd: 2, h: 0.043, Chi-2: 17.1690, P-value: 0.0707
OR, grd: 3, h: 0.057, Chi-2: 23.7898, P-value: 0.0688
OR, grd: 4, h: 0.035, Chi-2: 16.7021, P-value: 0.0812
ICU, grd: 1, h: 0.100, Chi-2: 23.9869, P-value: 0.0653
ICU, grd: 2, h: 0.063, Chi-2: 19.6595, P-value: 0.073

In [7]:
def save_dict_as_json(data_dict, filename, folder):
    os.makedirs(folder, exist_ok=True)  # Create folder if it doesn't exist
    path = os.path.join(folder, filename)
    with open(path, 'w') as f:
        json.dump(data_dict, f, indent=4)
    print(f"Dictionary saved to: {path}")

# Save the results to a JSON file
save_dict_as_json(resultados, filename="llegadas.json", folder="resultados incertidumbre")

Dictionary saved to: resultados incertidumbre/llegadas.json
